# TODO
1. extract relevant data from csv; create object
2. check for 

In [1]:
import csv
import re
import sqlite3

class Knight:
    def __init__(self, first_name, middle_name, last_name, wife, 
        address, city, zipcode, primary_phone, secondary_phone,
        email, council, assembly, role, deceased=0, state='Oklahoma'):
        """
        Initialize a Knight object
        """

        self.first_name     = first_name
        self.middle_name    = middle_name
        self.last_name      = last_name
        self.wife           = wife
        self.address        = address
        self.city           = city
        self.zipcode        = zipcode
        self.primary_phone  = primary_phone
        self.secondary_phone= secondary_phone
        self.email          = email
        self.council        = council
        self.deceased       = deceased
        self.state          = state
        self.assembly       = assembly
        self.role           = role

    @classmethod
    def create(cls, result: list):
        """
        Static method to create a knight object from a properly
        formatted row
        """
        d = {
             'OK': 'Oklahoma',
             'KS':'Kansas',
             'MO': 'Missouri',
             'Oklahoma': 'Oklahoma',
             'Kansas': 'Kansas'
            }

        k = Knight(
            first_name=f'{result[0]} {result[1]}' if result[0] != '' else result[1],
            middle_name=result[2],
            last_name=f'{result[3]} {result[4]}' if result[4] != '' else result[3],
            wife='',
            address=result[5],
            city=result[6],
            state=d[result[7]],
            zipcode=(result[8] if len(result[8]) == 5 else f'{result[8][0:5]}-{result[8][5:]}'),
            primary_phone='1' + re.sub('\D', '', result[9]) if result[9] != '' else '',             # type: ignore
            secondary_phone='1' + re.sub('\D', '', result[10]) if result[10] != '' else '',         # type: ignore
            email=result[11],
            council=result[12],
            assembly=result[13],
            role=result[14])
        
        return k
    
def get_csv_data(csvpath='./ok_council_list.csv'):
    data = []
    with open(csvpath, newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        for row in csvreader:
            data.append(row)

    return data

def get_db_data(dbpath='./ok_knights_directory.db'):
    data = []
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    
    query = "SELECT * FROM knights ORDER BY id"
    cursor.execute(query)
    rows = cursor.fetchall()

    for row in rows:
        data.append(row)

    conn.close()
    return data

csv_data = get_csv_data()
db_data = get_db_data()


In [2]:
# result = [csv_data[2][i] for i in [14,15,16,17,18,12,13,8,10,23,25,7,1,2,5,4]]

# result1 = [str.lower(csv_data[2][i]) for i in [14,15,16,17,18,1,2,5]]
# result2 = [db_data[10][i] or '' for i in [1,2,3]]

# print(result1)
# print(result2)

In [3]:
knights = []
for cd in csv_data[1:]:
    result = [cd[i] for i in [14,15,16,17,18,12,13,8,10,23,25,7,1,2,5,4]]
    knights.append(Knight.create(result))

In [4]:
dbpath='./ok_knights_directory.db'
conn = sqlite3.connect(dbpath)
cursor = conn.cursor()
cursor.execute('SELECT * from roles WHERE role in (\"Grand Knight\", \"Deputy Grand Knight\", \"Financial Secretary\")')
for row in cursor.fetchall():
    print(row)

(75, 'Grand Knight')
(76, 'Deputy Grand Knight')
(77, 'Financial Secretary')


In [10]:
roles = {
    'Grand Knight': 75,
    'Deputy Grand Knight': 76,
    'Financial Secretary': 77
}


BASE_QUERY = "INSERT INTO \"knights\" (first_name, middle_name, last_name, wife, address, city, zipcode, primary_phone, secondary_phone, email, deceased, state, council, assembly) VALUES "
KR_BASE_QUERY = "INSERT INTO \"knights_roles\" (knight_id, role_id) VALUES "

def insert_knights(knights_data: list[Knight], base_query):
    query = base_query
    for knight in knights_data:
        council = knight.council if knight.council != '' else 'NULL'
        assembly = knight.assembly if knight.assembly != '' else 'NULL'
        #role_id = roles[knight.role]
        query += f"('{knight.first_name}', '{knight.middle_name}', '{knight.last_name}', '{knight.wife}', '{knight.address}', '{knight.city}', '{knight.zipcode}', '{knight.primary_phone}', '{knight.secondary_phone}', '{knight.email}', {knight.deceased}, '{knight.state}', {council}, {assembly}),"
    return query

# update knights
query = insert_knights(knights, BASE_QUERY)[:-1]
dbpath='./ok_knights_directory.db'
conn = sqlite3.connect(dbpath)
cursor = conn.cursor()
cursor.execute(query)

# update councils
id = 544
for k in knights:
    if k.role == 'Grand Knight':
        cursor.execute(f"UPDATE 'councils' SET gk_id={id} WHERE council_number={k.council}")
    elif k.role == 'Financial Secretary':
        cursor.execute(f"UPDATE 'councils' SET fs_id={id} WHERE council_number={k.council}")
    id+=1

# really go
conn.commit()
conn.close()

In [ ]:
gk_id = 0
fs_id = 0
council = 0
COUNCIL_QUERY = f"UPDATE 'councils' SET gk_id={gk_id}, fs_id={fs_id} WHERE council_number={council}"